# Analisis de avistamientos ovni

## Dependencias

In [1]:
import pandas as pd
pd.set_option("display.max_columns",200)
pd.set_option("display.max_rows",15)

In [2]:
def completitud(df):
    comple=pd.DataFrame(df.isnull().sum())
    comple.reset_index(inplace=True)
    comple=comple.rename(columns={"index":"columna",0:"total"})
    comple["completitud"]=(1-comple["total"]/df.shape[0])*100
    comple=comple.sort_values(by="completitud",ascending=True)
    comple.reset_index(drop=True,inplace=True)
    return comple

## Lectura de los datos

In [3]:
df=pd.read_csv('data\complete.csv',encoding='UTF-8',error_bad_lines=False,warn_bad_lines=False,low_memory=False)
df.head(5)

E:\Users\1060240\AppData\Local\Temp\ipykernel_15772\866934925.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df=pd.read_csv('data\complete.csv',encoding='UTF-8',error_bad_lines=False,warn_bad_lines=False,low_memory=False)
E:\Users\1060240\AppData\Local\Temp\ipykernel_15772\866934925.py:1: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df=pd.read_csv('data\complete.csv',encoding='UTF-8',error_bad_lines=False,warn_bad_lines=False,low_memory=False)


,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667
3,10/10/1956 21:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833
4,10/10/1960 20:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611


In [14]:
df.shape

(88679, 11)

In [15]:
df.columns

Index(['datetime', 'city', 'state', 'country', 'shape', 'duration (seconds)',
       'duration (hours/min)', 'comments', 'date posted', 'latitude',
       'longitude'],
      dtype='object')

In [16]:
# Renombramos las variables
df = df.rename(columns={'datetime': 'occurance','shape':'ufoshape', 'state': 'citystate','duration (seconds)':'dseconds','duration (hours/min)':'duration','comments':'description','date posted':'report_date','latitude':'lat','longitude':'long'})
df.columns

Index(['occurance', 'city', 'citystate', 'country', 'ufoshape', 'dseconds',
       'duration', 'description', 'report_date', 'lat', 'long'],
      dtype='object')

In [17]:
df.head(1)

,occurance,city,citystate,country,ufoshape,dseconds,duration,description,report_date,lat,long
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111


## Eda

In [19]:
#Primero veamos la completid de nuestro dataset
completitud(df)

,columna,total,completitud
0,country,12365,86.056451
1,citystate,7409,91.645147
2,duration,3019,96.595586
3,ufoshape,2922,96.704970
4,description,35,99.960532
5,dseconds,2,99.997745
6,occurance,0,100.000000
7,city,0,100.000000
8,report_date,0,100.000000
9,lat,0,100.000000


In [21]:
#Observamos que, en general, el dataset conserva buena completitud en sus variables, se explorará la posibilidad
# de imputar los valores faltantes en las variables respectivas

In [22]:
#Demos un vistazo rapido a nuestros datos
df.head()

,occurance,city,citystate,country,ufoshape,dseconds,duration,description,report_date,lat,long
0,10/10/1949 20:30,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111
1,10/10/1949 21:00,lackland afb,tx,NaN,light,7200,1-2 hrs,1949 Lackland AFB&#44 TX. Lights racing acros...,12/16/2005,29.38421,-98.581082
2,10/10/1955 17:00,chester (uk/england),NaN,gb,circle,20,20 seconds,Green/Orange circular disc over Chester&#44 En...,1/21/2008,53.2,-2.916667
3,10/10/1956 21:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833
4,10/10/1960 20:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611


In [27]:
#Ahora demos un vistazo a la cantidad de datos duplicados en el dataset
df.duplicated().value_counts()
#Ahora sabemos que no hay registros duplicados dentro de nuestro conjunto de datos

False    88679
dtype: int64

In [30]:
df.ufoshape.value_counts(1)

light       0.208403
triangle    0.098989
circle      0.098569
fireball    0.076519
unknown     0.073685
              ...   
pyramid     0.000012
flare       0.000012
hexagon     0.000012
dome        0.000012
changed     0.000012
Name: ufoshape, Length: 29, dtype: float64